In [2]:
# Check core SDK version number for debugging purposes
import azureml.core
print("SDK Version:", azureml.core.VERSION)

SDK Version: 0.1.0.995245


In [3]:
subscription_id = "786d6510-8f1e-4ae7-b55b-5178716e6ac8"
resource_group = "test-anivan"
workspace_name = "workspace-test1"
workspace_region = 'eastus2' # or eastus2euap

In [4]:
# import the Workspace class and check the azureml SDK version# impor 
from azureml.core.workspace import Workspace, WorkspaceException

ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region)
ws.get_details()

Performing interactive authentication. Please follow the instructions on the terminal.


To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FZKD6GMH9 to authenticate.


Interactive authentication successfully completed.


{'applicationInsights': '/subscriptions/786d6510-8f1e-4ae7-b55b-5178716e6ac8/resourcegroups/test-anivan/providers/microsoft.insights/components/workspacinsightslksuficz',
 'containerRegistry': '/subscriptions/786d6510-8f1e-4ae7-b55b-5178716e6ac8/resourcegroups/test-anivan/providers/microsoft.containerregistry/registries/workspacacrezbodeet',
 'description': '',
 'friendlyName': 'workspace-test1',
 'id': '/subscriptions/786d6510-8f1e-4ae7-b55b-5178716e6ac8/resourceGroups/test-anivan/providers/Microsoft.MachineLearningServices/workspaces/workspace-test1',
 'identityPrincipalId': '2059178a-03cf-4cc9-ad26-a5d8ba8ee61b',
 'identityTenantId': '72f988bf-86f1-41af-91ab-2d7cd011db47',
 'identityType': 'SystemAssigned',
 'keyVault': '/subscriptions/786d6510-8f1e-4ae7-b55b-5178716e6ac8/resourcegroups/test-anivan/providers/microsoft.keyvault/vaults/workspackeyvaultqkjjihff',
 'location': 'eastus2',
 'name': 'workspace-test1',
 'storageAccount': '/subscriptions/786d6510-8f1e-4ae7-b55b-5178716e6ac8/

In [6]:
model = ws.register_model(model_path = "model.pkl",
                          model_name = "mymodel",
                          tags = ["pdm"],
                          description = "Predictive Maintenance multi-class classifier")

Registering model: mymodel


In [7]:
%%writefile score.py

import pickle
import json
import numpy
from sklearn.externals import joblib
from sklearn.linear_model import Ridge

def init():
    global model
    model = joblib.load("model.pkl")

def run(rawdata):
    try:
        data = json.loads(rawdata)['data']
        data = numpy.array(data).reshape(1,len(data))
        result = model.predict(data)[0]
    except Exception as e:
        result = str(e)
    return json.dumps({"result":result})

Writing score.py


In [8]:
%%writefile env.yml

name: myenv
channels:
  - defaults
dependencies:
  - pip:
    - numpy==1.14.2

Writing env.yml


In [10]:
image = ws.create_image(name = "myimage",
                        models = [model],
                        runtime = "python",
                        driver_file = "score.py",
                        conda_file = "env.yml",
                        tags = ["pdm"],
                        description = "Image for Predicitve Maintenance multi-class classifier")

Creating image


In [11]:
image.wait_for_creation(show_output = True)

Running..................................
SucceededImage creation operation finished for image myimage:2, operation "Succeeded"


In [12]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = ['pdm'], 
                                               description = 'Predicting equipment failure')

In [13]:
service = ws.deploy_webservice(name = 'myaci',
                               deploy_config = aciconfig,
                               target = None,
                               image = image)
service.wait_for_deployment(show_output = True)

Creating service
Running...................................

WebserviceException: Received bad response from Resource Provider:
Response Code: 500
Headers: {'Content-Type': 'application/json', 'Connection': 'keep-alive', 'x-ms-client-request-id': 'f50c45475798463ba60bfac8326346ff', 'Date': 'Mon, 04 Jun 2018 23:45:57 GMT', 'x-ms-client-session-id': '', 'api-supported-versions': '2018-03-01-preview', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload', 'Transfer-Encoding': 'chunked'}
Content: b'{"code":"InternalServerError","statusCode":500,"message":"An internal server error occurred. Please try again. If the problem persists, contact support"}'

In [15]:
service = ws.get_webservice(name = 'myaci')
import json

test_sample = json.dumps({'data': [
    [1,2,3,4,5,6,7,8,9,10], 
    [10,9,8,7,6,5,4,3,2,1]
]})
test_sample = bytes(test_sample,encoding = 'utf8')

prediction = service.run(input_data = test_sample)
print(prediction)

{"result": "cannot reshape array of size 20 into shape (1,2)"}
